In [1]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd
import random

In [2]:
frame = pd.DataFrame(random.randint(1,9), range(3), range(3))

In [3]:
frame.at[1,2] = 10

In [4]:
frame

,0,1,2
0,7,7,7
1,7,7,10
2,7,7,7


In [5]:
frame.at[1,2]

10

In [6]:
Box(int(1),int(9),shape=(3,3), dtype = int).sample()

array([[6, 8, 4],
       [2, 7, 2],
       [9, 2, 6]])

In [7]:
frame[1][0] = 5

In [8]:
frame

,0,1,2
0,7,5,7
1,7,7,10
2,7,7,7


In [9]:
MultiDiscrete([3,3]).sample()

array([2, 2])

In [10]:
lijst = [1,1,1,2,2,2,3,3,3]
lijst = random.shuffle(lijst)

In [11]:
def getListOfContainers(numberOfShips, containersPerShip):
    containers = list(range(1, numberOfShips+1))*containersPerShip
    containers.sort()
    random.shuffle(containers)
    return containers

In [12]:
getListOfContainers(3,3)

[3, 2, 2, 3, 2, 1, 1, 3, 1]

In [13]:
pd.DataFrame(index=range(3),columns=range(3)).fillna(0)

,0,1,2
0,0,0,0
1,0,0,0
2,0,0,0


In [14]:
###
# observation naar dictionary, {box, discrete}
# 

In [15]:
class Container(Env):
    def __init__(self):
        self.state = pd.DataFrame(index=range(3),columns=range(3)).fillna(0)
        self.observation_space = Dict(spaces ={
            'grid': Box(int(0),int(9),shape=(3,3), dtype = int),
            'container': Discrete(int(4)),
        })
        
        Box(int(0),int(9),shape=(3,3), dtype = int) 
        self.action_space = MultiDiscrete([3,3])
        self.list_containers = getListOfContainers(3,3)
        
    def step(self, action):
        container = self.list_containers.pop()
        preview_cont = self.state.at[action[0], action[1]]
        
        if preview_cont == 0: #als de plek leeg is; reward = 1
            reward = 1
        else:
            reward = -2
            
        if action[1] == 1:
            if self.state.at[action[0],action[1]-1] and self.state.at[action[0],action[1]+1] != 0:
                reward = -2
                
        if action[1] == 0: #container wordt geplaatst in de linker kolom
            if self.state.at[action[0], action[1]+1] == container: #container rechts van plaatsing voor zelfde schip?; +1
                reward += 1
                
        if action[1] == 2: #als container wordt geplaatst in de rechter kolom
            if self.state.at[action[0], action[1]-1] == container:
                reward += 1
        

        
        self.state.at[action[0],action[1]] = container #plaatsen
        
        if len(self.list_containers) == 0: #als lijst leeg is stopt het
            obs = {'container': np.nan, 'grid': self.state.to_numpy()}
            done = True
        else:
            obs = {'container': self.list_containers[-1], 'grid': self.state.to_numpy()}
            done = False
            
        info = {}
                
        return obs, reward, done, info
    
    def render(self, mode= 'human', close = False):
        display(self.state)
        
    
    def reset(self):
        self.state = pd.DataFrame(index=range(3),columns=range(3)).fillna(0)
        self.list_containers = getListOfContainers(3,3)
        obs = {'container': self.list_containers[-1], 'grid': self.state.to_numpy()}
        return obs

In [16]:
env = Container()

In [17]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

,0,1,2
0,0,0,0
1,0,0,0
2,0,0,0


,0,1,2
0,0,0,0
1,0,0,1
2,0,0,0


,0,1,2
0,0,0,0
1,0,0,3
2,0,0,0


,0,1,2
0,3,0,0
1,0,0,3
2,0,0,0


,0,1,2
0,3,0,1
1,0,0,3
2,0,0,0


...

,0,1,2
0,2,0,0
1,1,0,3
2,0,3,0


,0,1,2
0,2,0,0
1,2,0,3
2,0,3,0


,0,1,2
0,2,0,0
1,2,2,3
2,0,3,0


,0,1,2
0,2,0,0
1,2,1,3
2,0,3,0


Episode:5 Score:-3


In [18]:
model = PPO("MultiInputPolicy", env, verbose=0)

/opt/jupyterhub/anaconda/lib/python3.9/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484809662/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [19]:
model.learn(total_timesteps= 100000)

In [20]:
from stable_baselines3.common.monitor import Monitor
env.reset()
monitor = Monitor(env)
evaluate_policy(model, monitor, n_eval_episodes=1, return_episode_rewards= True, render=True)

,0,1,2
0,0,0,0
1,0,1,0
2,0,0,0


,0,1,2
0,0,2,0
1,0,1,0
2,0,0,0


,0,1,2
0,0,2,0
1,0,1,0
2,0,3,0


,0,1,2
0,0,2,0
1,0,1,0
2,0,3,3


,0,1,2
0,0,2,0
1,1,1,0
2,0,3,3


,0,1,2
0,2,2,0
1,1,1,0
2,0,3,3


,0,1,2
0,2,2,0
1,1,1,1
2,0,3,3


,0,1,2
0,2,2,2
1,1,1,1
2,0,3,3


,0,1,2
0,0,0,0
1,0,0,0
2,0,0,0


([15], [9])